In [41]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import SGDRegressor, LinearRegression, Ridge
from sklearn import metrics
%matplotlib inline

import warnings
def ignore_warn(*args, **kwargs):
    pass
warnings.warn = ignore_warn

In [42]:
data = pd.read_csv('./House_Rent_Dataset.csv')
data.head()

,Posted On,BHK,Rent,Size,Floor,Area Type,Area Locality,City,Furnishing Status,Tenant Preferred,Bathroom,Point of Contact
0,2022-05-18,2,10000,1100,Ground out of 2,Super Area,Bandel,Kolkata,Unfurnished,Bachelors/Family,2,Contact Owner
1,2022-05-13,2,20000,800,1 out of 3,Super Area,"Phool Bagan, Kankurgachi",Kolkata,Semi-Furnished,Bachelors/Family,1,Contact Owner
2,2022-05-16,2,17000,1000,1 out of 3,Super Area,Salt Lake City Sector 2,Kolkata,Semi-Furnished,Bachelors/Family,1,Contact Owner
3,2022-07-04,2,10000,800,1 out of 2,Super Area,Dumdum Park,Kolkata,Unfurnished,Bachelors/Family,1,Contact Owner
4,2022-05-09,2,7500,850,1 out of 2,Carpet Area,South Dum Dum,Kolkata,Unfurnished,Bachelors,1,Contact Owner


### Предварительная обработка данных
В первую очередь проверим есть ли пропущенные значения

In [43]:
data.isna().sum()

Posted On            0
BHK                  0
Rent                 0
Size                 0
Floor                0
Area Type            0
Area Locality        0
City                 0
Furnishing Status    0
Tenant Preferred     0
Bathroom             0
Point of Contact     0
dtype: int64

Пропущенных значений нет, 

In [44]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4746 entries, 0 to 4745
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Posted On          4746 non-null   object
 1   BHK                4746 non-null   int64 
 2   Rent               4746 non-null   int64 
 3   Size               4746 non-null   int64 
 4   Floor              4746 non-null   object
 5   Area Type          4746 non-null   object
 6   Area Locality      4746 non-null   object
 7   City               4746 non-null   object
 8   Furnishing Status  4746 non-null   object
 9   Tenant Preferred   4746 non-null   object
 10  Bathroom           4746 non-null   int64 
 11  Point of Contact   4746 non-null   object
dtypes: int64(4), object(8)
memory usage: 445.1+ KB


In [45]:
# у столбца Posted On указан тип object, преобразуем его во время.
data['Posted On'] = pd.to_datetime(data['Posted On'])
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4746 entries, 0 to 4745
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   Posted On          4746 non-null   datetime64[ns]
 1   BHK                4746 non-null   int64         
 2   Rent               4746 non-null   int64         
 3   Size               4746 non-null   int64         
 4   Floor              4746 non-null   object        
 5   Area Type          4746 non-null   object        
 6   Area Locality      4746 non-null   object        
 7   City               4746 non-null   object        
 8   Furnishing Status  4746 non-null   object        
 9   Tenant Preferred   4746 non-null   object        
 10  Bathroom           4746 non-null   int64         
 11  Point of Contact   4746 non-null   object        
dtypes: datetime64[ns](1), int64(4), object(7)
memory usage: 445.1+ KB


В столбце `Floor` указано как общее количество этажей, так и сам этаж апартаментов. Разобьем эту составляющую на два отдельных столбца с данными.

In [46]:
# data["Total Floors"] = data["Floor"].apply(lambda floor:floor.split()[-1])
# data["Floor"] = data["Floor"].apply(lambda floor:floor.split()[0])

# encodes = {"Floor":     {"Ground": 0, "Upper": 999, "Lower": -1},
#            "Total Floors": {"Ground": 0}}

# data = data.replace(encodes)

# data.head()

In [47]:
[(data.columns[i], data.iloc[:, i].unique().shape[0]) for i in range(len(data.columns))]

[('Posted On', 81),
 ('BHK', 6),
 ('Rent', 243),
 ('Size', 615),
 ('Floor', 480),
 ('Area Type', 3),
 ('Area Locality', 2235),
 ('City', 6),
 ('Furnishing Status', 3),
 ('Tenant Preferred', 3),
 ('Bathroom', 8),
 ('Point of Contact', 3)]

В столбце `Area Locality`, где хранится информация о местоположении дома, слишком много уникальных значений

In [48]:
data = data.drop(['Area Locality', 'Posted On', 'Floor'],axis=1)

In [49]:
columns = ['Area Type', 'City', 'Furnishing Status', 'Tenant Preferred', 'Point of Contact']
data = pd.get_dummies(data, columns=columns)

data.head()

,BHK,Rent,Size,Bathroom,Area Type_Built Area,Area Type_Carpet Area,Area Type_Super Area,City_Bangalore,City_Chennai,City_Delhi,...,City_Mumbai,Furnishing Status_Furnished,Furnishing Status_Semi-Furnished,Furnishing Status_Unfurnished,Tenant Preferred_Bachelors,Tenant Preferred_Bachelors/Family,Tenant Preferred_Family,Point of Contact_Contact Agent,Point of Contact_Contact Builder,Point of Contact_Contact Owner
0,2,10000,1100,2,False,False,True,False,False,False,...,False,False,False,True,False,True,False,False,False,True
1,2,20000,800,1,False,False,True,False,False,False,...,False,False,True,False,False,True,False,False,False,True
2,2,17000,1000,1,False,False,True,False,False,False,...,False,False,True,False,False,True,False,False,False,True
3,2,10000,800,1,False,False,True,False,False,False,...,False,False,False,True,False,True,False,False,False,True
4,2,7500,850,1,False,True,False,False,False,False,...,False,False,False,True,True,False,False,False,False,True


In [50]:

data.corr()['Rent'].sort_values(ascending=False)

Rent                                 1.000000
Bathroom                             0.441215
Size                                 0.413551
BHK                                  0.369718
Point of Contact_Contact Agent       0.339750
City_Mumbai                          0.327038
Area Type_Carpet Area                0.215769
Furnishing Status_Furnished          0.110576
Tenant Preferred_Family              0.063941
Furnishing Status_Semi-Furnished     0.045309
Tenant Preferred_Bachelors           0.042151
Point of Contact_Contact Builder    -0.005482
Area Type_Built Area                -0.006439
City_Delhi                          -0.027072
City_Bangalore                      -0.061512
Tenant Preferred_Bachelors/Family   -0.078774
City_Chennai                        -0.082361
City_Hyderabad                      -0.087465
City_Kolkata                        -0.105322
Furnishing Status_Unfurnished       -0.126271
Area Type_Super Area                -0.215499
Point of Contact_Contact Owner    

### Разбиваем данные и масштабируем данные

In [51]:
# Разделение на признаки и целевую переменную
X = data.drop('Rent', axis=1)
y = data['Rent']

# Разделение на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [52]:
# Scaling the data
y_train = y_train.values.reshape(-1,1)
y_tes = y_test.values.reshape(-1,1)
scaler = StandardScaler()
sc_y = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

### Моделирование

In [53]:
from numpy import ndarray
# Модель линейной регрессии без регуляризации
linear_reg = LinearRegression()
linear_reg.fit(X_train, y_train)

# Модель линейной регрессии с L2-регуляризацией (Ridge)
ridge_reg = Ridge(alpha=0.5)
ridge_reg.fit(X_train, y_train)


linear_reg_prediction = linear_reg.predict(X_test)
ridge_reg_prediction = ridge_reg.predict(X_test)

def print_metrics(prediction: ndarray):
    # Evaluation metrics
    mae = metrics.mean_absolute_error(y_test, prediction)
    mse=  metrics.mean_squared_error(y_test, prediction)
    rmse =  np.sqrt(mse)
    print('MAE:', mae)
    print('MSE:', mse)
    print('RMSE:', rmse)
    print('\n')

print("Значения функции потерь (линейная регрессия):")
print_metrics(linear_reg_prediction)
print("Значения функции потерь (регуляризация Ridge):")
print_metrics(ridge_reg_prediction)

Значения функции потерь (линейная регрессия):
MAE: 6196861716843749.0
MSE: 1.1356299080116221e+32
RMSE: 1.0656593771049088e+16


Значения функции потерь (регуляризация Ridge):
MAE: 22316.591562243033
MSE: 1624280058.254136
RMSE: 40302.35797387215




Получим модель линейной регрессии с помощью стохастического градиентного спуска

In [54]:
sgdr_reg = SGDRegressor(tol=.0001, eta0=.01) 
sgdr_reg.fit(X_train, y_train)
sgdr_reg_prediction = sgdr_reg.predict(X_test)
print("Значения функции потерь (Стохастический градиентный спуск):")
print_metrics(sgdr_reg_prediction)

Значения функции потерь (Стохастический градиентный спуск):
MAE: 25933.795851107425
MSE: 1800360626.970854
RMSE: 42430.6566879521




### Какая модель имеет наименьшее значение функции потерь на тестовой выборке?

**Наименьшее значение функции потерь на тестовой выборке имеет модель с регуляризацией L2, следовательно, она имеет наилучшую точность**


Регуляризация помогает избежать эффекта переобучения в данном примере. Использование L2-регуляризации (Ridge) позволяет штрафовать большие значения коэффициентов модели, что помогает уменьшить переобучение.

### Какие признаки оказывают наибольший вклад в точность определения стоимости аренды?

Чтобы определить признаки, которые оказывают наибольший вклад в точность определения стоимости аренды, можно использовать коэффициенты модели линейной регрессии. Чем больше абсолютное значение коэффициента, тем больший вклад вносит соответствующий признак.

In [55]:
# Выведем все коэффициенты от самого высокого по модулю до самого низкого (т.е. от более значимых до менее значимых) для каждой из представленных моделей
feature_importance = pd.DataFrame({"Признак": X.columns, "Коэффициент": linear_reg.coef_[0]})
feature_importance_sorted = feature_importance.iloc[feature_importance['Коэффициент'].abs().argsort()[::-1]]
print("Наиболее важные признаки для линейной регрессии:")
print(feature_importance_sorted)

Наиболее важные признаки для линейной регрессии:
                              Признак   Коэффициент
16  Tenant Preferred_Bachelors/Family  3.864205e+17
15         Tenant Preferred_Bachelors  3.274686e+17
17            Tenant Preferred_Family  2.616761e+17
13   Furnishing Status_Semi-Furnished -6.771444e+16
14      Furnishing Status_Unfurnished -6.592735e+16
12        Furnishing Status_Furnished -4.712783e+16
11                        City_Mumbai -2.059286e+16
7                        City_Chennai -2.010158e+16
6                      City_Bangalore -2.004090e+16
9                      City_Hyderabad -1.969210e+16
8                          City_Delhi -1.704134e+16
10                       City_Kolkata -1.579364e+16
20     Point of Contact_Contact Owner  9.954992e+15
18     Point of Contact_Contact Agent  9.952574e+15
5                Area Type_Super Area -9.270821e+15
4               Area Type_Carpet Area -9.270538e+15
3                Area Type_Built Area -4.549513e+14
19   Point of C

In [56]:

feature_importance = pd.DataFrame({"Признак": X.columns, "Коэффициент": ridge_reg.coef_[0]})
feature_importance_sorted = feature_importance.iloc[feature_importance['Коэффициент'].abs().argsort()[::-1]]
print("Наиболее важные признаки для Ridge:")
print(feature_importance_sorted)

Наиболее важные признаки для Ridge:
                              Признак   Коэффициент
1                                Size  23789.167440
11                        City_Mumbai  17811.922994
2                            Bathroom  10243.462744
9                      City_Hyderabad  -9039.153338
7                        City_Chennai  -5394.706110
0                                 BHK   3244.215468
17            Tenant Preferred_Family  -2977.460734
10                       City_Kolkata  -2737.456455
12        Furnishing Status_Furnished   2303.564654
20     Point of Contact_Contact Owner  -2288.890348
18     Point of Contact_Contact Agent   2270.256426
6                      City_Bangalore  -2155.025911
15         Tenant Preferred_Bachelors   1304.968560
13   Furnishing Status_Semi-Furnished   -989.449892
16  Tenant Preferred_Bachelors/Family    910.392313
14      Furnishing Status_Unfurnished   -630.420609
5                Area Type_Super Area   -607.685287
4               Area Type_Ca

In [57]:

feature_importance = pd.DataFrame({"Признак": X.columns, "Коэффициент": sgdr_reg.coef_})
feature_importance_sorted = feature_importance.iloc[feature_importance['Коэффициент'].abs().argsort()[::-1]]
print("Наиболее важные признаки для градиентного спуска:")
print(feature_importance_sorted)

Наиболее важные признаки для градиентного спуска:
                              Признак    Коэффициент
19   Point of Contact_Contact Builder  361709.897096
1                                Size   25298.405296
11                        City_Mumbai   13947.861067
20     Point of Contact_Contact Owner  -12836.855921
2                            Bathroom   10488.309599
7                        City_Chennai   -6273.132522
10                       City_Kolkata   -4176.613192
9                      City_Hyderabad   -3970.880955
17            Tenant Preferred_Family   -3269.899076
15         Tenant Preferred_Bachelors    3066.759563
0                                 BHK    2596.036387
12        Furnishing Status_Furnished    1819.743130
8                          City_Delhi   -1668.552485
14      Furnishing Status_Unfurnished   -1256.977490
3                Area Type_Built Area     911.177997
18     Point of Contact_Contact Agent    -575.145293
6                      City_Bangalore     572.173